In [ ]:
import torchvision
import torch
import sys

sys.path.append("..")

from torch.utils.data import DataLoader, Dataset
from utils.model_util import LeNet5, CNN, MLP
from utils.train_util import *

model = LeNet5(28, 28, 1, 10).cuda()
train_dataset_loader = DataLoader(
    dataset=torchvision.datasets.MNIST(
        root='../data/raw-data/',
        train=True,
        transform=torchvision.transforms.ToTensor()),
    batch_size=600,
    shuffle=False)
test_dataset_loader = DataLoader(
    dataset=torchvision.datasets.mnist.MNIST(
        root='../data/raw-data/',
        transform=torchvision.transforms.ToTensor()),
    batch_size=32,
    shuffle=False)
test_dataset = torchvision.datasets.mnist.MNIST(
    root='../data/raw-data/',
    transform=torchvision.transforms.ToTensor())

opti = torch.optim.Adam(model.parameters())
criterion = torch.nn.CrossEntropyLoss(reduction='sum')
for data, target in train_dataset_loader:
    opti.zero_grad()
    output = model(data.cuda())
    loss = criterion(output, target.cuda())
    loss.backward()
    print(loss.item())
    opti.step()

print(loss, type(loss))
print(loss.item(), type(loss.item()))

model = LeNet5(28, 28, 1, 10).cuda()
train_dataset_loader = DataLoader(
    dataset=torchvision.datasets.MNIST(
        root='../data/raw-data/',
        train=True,
        transform=torchvision.transforms.ToTensor()),
    batch_size=600,
    shuffle=False)
test_dataset_loader = DataLoader(
    dataset=torchvision.datasets.mnist.MNIST(
        root='../data/raw-data/',
        transform=torchvision.transforms.ToTensor()),
    batch_size=32,
    shuffle=False)

opti = torch.optim.Adam(model.parameters())
criterion = torch.nn.CrossEntropyLoss(reduction='sum')
loss_sum = 0
for data, target in train_dataset_loader:
    opti.zero_grad()
    output = model(data.cuda())
    loss = criterion(output, target.cuda())
    # print(f'loss_sum = {loss_sum:<20}', end=' | ')
    loss.backward()
    loss_sum += loss.item()
    # print(f'loss = {loss.item():<20}')
    opti.step()

def eval_model(model, dataset, device):
    '''
    评估模型
    '''
    model_copy = deepcopy(model).eval()
    correct = 0
    data_loader = DataLoader(dataset, batch_size=32)
    for images, targets in data_loader:
        outputs = model_copy(images.to(device))
        _, predicted = torch.max(outputs, 1)
        correct += torch.eq(predicted, targets.to(device)).sum()
    # print(f'Test Accuracy: {100 * correct / total:.2f}%')
    accuracy = correct / len(dataset)
    return accuracy

eval_model(model, test_dataset, 'cuda')

model_copy = deepcopy(model).eval()
model_copy.eval()
model_copy.cuda()
correct = 0
total = 0
data_loader = test_dataset_loader
for images, targets in data_loader:
    outputs = model_copy(images.cuda())
    _, predicted = torch.max(outputs.data, 1)
    total += targets.size(0)
    correct += (predicted == targets.cuda()).sum().item()
    break
print('Test Accuracy: {:.2f}%'.format(100 * correct / total))

In [ ]:
for i in range(8, 13):
    print(f'{i:<10}|')
    print(f'{i:.2f}|')

In [ ]:
def get_list(len_list, term=[]):
    '''
    返回一个全是空列表的列表
    '''
    list_return = []
    for _ in range(len_list):
        list_return.append(term)
    return list_return

num_all_client = 6
num_all_server = 2

server_model = get_list(num_all_server)
server_accuracy = get_list(num_all_server)
client_accuracy = get_list(num_all_client)
server_client_model = get_list(num_all_server)
server_model_distillation_accuracy = get_list(num_all_server)

print(client_accuracy)

In [ ]:
import torch

from utils.data_util import *
from utils.train_util import *
from utils.model_util import *
from utils.lib_util import *

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
print(device)

save_path = './data/dealed-data/'
train_dataset_o, test_dataset_o = get_dataset()

model = LeNet5(28, 28, 1, 10)
